# 03 - Vector Search Homework

## Q1. Getting the embeddings model

In [25]:
from tqdm.auto import tqdm
import numpy as np

In [1]:
model_name = 'multi-qa-distilbert-cos-v1'

In [2]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

/home/onwunalu/.pyenv/versions/llm-zoomcamp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
user_question = "I just discovered the course. Can I still join it?"


In [5]:
user_question_vector = embedding_model.encode(user_question)
user_question_vector[0]

0.078222655

### Q1b: Prepare the documents

In [6]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [8]:
# filter the document
documents_filtered = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']

In [9]:
len(documents_filtered)

375

In [ ]:
"""
Create a list embeddings
Iterate over each document
qa_text = f'{question} {text}'
compute the embedding for qa_text, append to embeddings
At the end, let X = np.array(embeddings) (import numpy as np)
"""

In [26]:
embeddings = []
for doc in tqdm(documents_filtered):
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    embeddings.append(embedding_model.encode(qa_text))


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:24<00:00, 15.10it/s]


In [29]:
X = np.array(embeddings)
X.shape

(375, 768)

# Q3. Search

In [87]:
scores = X.dot(user_question_vector)

In [88]:
max(scores)

0.6506571

## Vector search

In [121]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents_filtered, embeddings=X)
search_engine.search(user_question_vector, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693',
  'question_text_vector': array([ 8.06286484e-02, -6.66388869e-02,  2.52730995e-02, -1.30044194e-02,
          7.58728236e-02, -5.94626963e-02, -2.18838602e-02,  2.90002022e-03,
          7.92863546e-04, -5.22238575e-03, -3.36518176e-02, -2.79135015e-02,
          5.81165738e-02,  3.97483520e-02,  5.44185415e-02, -3.82517278e-02,
          6.30573854e-02, -3.89698818e-02,  4.33633220e-04, -3.55845131e-02,
          1.75641559e-03

# Q4. Hit-rate for our search engine

In [122]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [123]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

def new_vector_search_wrapper(doc):
    question = doc['question']
    query_vector = embedding_model.encode(question)
    results = search_engine.search(query_vector, num_results=5)
    return results
    

In [126]:
evaluate(ground_truth, new_vector_search_wrapper)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [03:37<00:00,  8.43it/s]


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}

# Q5. Indexing with Elasticsearch

In [49]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [48]:
# add the computed vectors to the document

In [51]:
for i, doc in tqdm(enumerate(documents_filtered)):
    doc['question_text_vector'] = X[i,:]

375it [00:00, 552269.66it/s]


In [52]:
# index the documents with the vectors
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:19<00:00, 48.67it/s]


In [127]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

def question_text_vector_knn(doc):
    question = doc['question']

    v_q = embedding_model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course='machine-learning-zoomcamp')

In [128]:
# Get the id of the top document returned for the query in Question 1
results = elastic_search_knn('question_text_vector', user_question_vector, course='machine-learning-zoomcamp')
results[0]['id']

'ee58a693'

In [ ]:
# id of the top document returned is: 'id': 'ee58a693'

# Q6. Hit-rate for Elasticsearch

In [129]:
evaluate(ground_truth, question_text_vector_knn)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [00:48<00:00, 37.75it/s]


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}